In [167]:
import numpy as np
import pandas as pd
import xlrd
pd.options.mode.chained_assignment = None  # default='warn'
#pd.set_option('display.max_rows',6)

regUsr_dat = pd.read_excel("2018 summary.xlsx", sheet_name=0)
unregUsr_dat = pd.read_excel("2018 summary.xlsx", sheet_name=1)
articles = pd.read_excel("2018 summary.xlsx", sheet_name="Articles")

# Introduction

The dataset comes from our third party content service provider, it contains the contact information of the user that have read our website articles, as well as their activities on our website.  

In this report, I will summarize the user activities and articles read, in order to identify the main areas of interest of the users, as well as the performance of our articles. 

# User Activities Dataset

In [168]:
# registered user dataset info
regUsr_dat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 557 entries, 0 to 556
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Ind_Title               447 non-null    object        
 1   Forename                547 non-null    object        
 2   Surname                 547 non-null    object        
 3   Email                   547 non-null    object        
 4   Company                 556 non-null    object        
 5   Industry                557 non-null    object        
 6   Country                 557 non-null    object        
 7   Job Role                557 non-null    object        
 8   Date Of Activity        557 non-null    datetime64[ns]
 9   Activity Type           557 non-null    object        
 10  Article Title           556 non-null    object        
 11  Article Primary Author  556 non-null    object        
 12  Primary Topic           556 non-null    object    

In [169]:
# select columns of interest
cols = ["Article Title", "Activity Type", "Date Of Activity", "Article Primary Author"]
regUsr_dat = regUsr_dat[cols]
regUsr_dat.head(1)

,Article Title,Activity Type,Date Of Activity,Article Primary Author
0,A Successful Trade Dress Protection Case Throu...,Article Read from Email Subscription,2018-02-26,Dan Chen


In [170]:
# unregistered user dataset info
unregUsr_dat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10402 entries, 0 to 10401
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Search Term         0 non-null      float64       
 1   Channel             3253 non-null   object        
 2   Article Title       10402 non-null  object        
 3   Activity Type       10402 non-null  object        
 4   Primary Author      10349 non-null  object        
 5   Date User Searched  10402 non-null  datetime64[ns]
 6   Primary Topic       10349 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(5)
memory usage: 569.0+ KB


In [171]:
# select the same columns for unregistered users
unregUsr_dat.rename(columns={"Date User Searched": "Date Of Activity",
                             "Primary Author": "Article Primary Author"}, inplace=True)
unregUsr_dat = unregUsr_dat[cols]

In [172]:
# check if columns of both dataset match now
unregUsr_dat.columns == regUsr_dat.columns

array([ True,  True,  True,  True])

In [173]:
# Top 10 articles read by registered users
regUsr_dat["Article Title"].value_counts()[:10].to_frame()

,Article Title
A Successful Trade Dress Protection Case Through Copyright And Unfair Competition Dispute Litigation By Unitalen,90
Revision Of Anti-Unfair Competition Law Of China,88
E-Commerce Law of China Will Come into Force in January 2019,62
Fast Growth of China IP License Trade in the First Half of 2018,35
"Two Unitalen Cases Selected among ""The 50 Typical IP Cases"" by China Supreme Court",35
"Chinese ""Lafite"" Mark Recognized as an Unregistered Well-known Mark by Shanghai IP Court",27
"Contributor Page View from: Unitalen Obtains Recognition Of Well-Known Mark For ""SISLEY希思黎"" Through Litigation",22
SIPO Changes English Translation to CNIPA,20
BSA: China Found with the Biggest Fall in Piracy Rate,18
TRAB: Simplified Document Requirement for Trademark Refusal Review,17


In [174]:
# Top 10 articles read by unregistered users
unregUsr_dat["Article Title"].value_counts()[:10].to_frame()

,Article Title
"Two Unitalen Cases Selected among ""The 50 Typical IP Cases"" by China Supreme Court",1394
"Chinese ""Lafite"" Mark Recognized as an Unregistered Well-known Mark by Shanghai IP Court",1048
SIPO Changes English Translation to CNIPA,584
BSA: China Found with the Biggest Fall in Piracy Rate,564
Obtaining a Cambodian Patent by Registration of Chinese Patent in Cambodia,531
A Successful Trade Dress Protection Case Through Copyright And Unfair Competition Dispute Litigation By Unitalen,449
Unitalen Helped Client Achieved Settlement in Patent Infringement Suit,447
Unitalen Client In Two-Dimensional Code Patent Invalidation Administrative Litigation,443
Revision Of Anti-Unfair Competition Law Of China,438
E-Commerce Law of China Will Come into Force in January 2019,420


# Articles dataset

In [175]:
# articles dataset info
articles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Article Title      32 non-null     object
 1   Article Type       32 non-null     object
 2   Matter of Concern  32 non-null     object
dtypes: object(3)
memory usage: 896.0+ bytes


In [176]:
articles['Article Title'] = articles['Article Title'].str.strip()
regUsr_dat['Article Title'] = regUsr_dat['Article Title'].str.strip()
unregUsr_dat['Article Title'] = unregUsr_dat['Article Title'].str.strip()

In [177]:
# check how many unique articles found in user activities
len(regUsr_dat['Article Title'].unique()), len(unregUsr_dat['Article Title'].unique())

(24, 108)

It's wierd that the number of unique articles in unregistered user dataset is much more than the articles we have. Let's explore where does this problem come from.

In [178]:
# show article titles that are not found in `aritles` dataset
uniqueArticles = pd.Series(unregUsr_dat['Article Title'].unique())
uniqueArticles[~uniqueArticles.isin(articles['Article Title'])].to_list()[:10] # limit to first 10 results

['Author Page View Unitalen Attorneys At Law',
 'Redirect to your Website from: Chinese "Lafite" Mark Recognized as an Unregistered Well-known Mark by Shanghai IP Court',
 'Contributor Page View from: Chinese "Lafite" Mark Recognized as an Unregistered Well-known Mark by Shanghai IP Court',
 'Press Release View from: BSA: China Found with the Biggest Fall in Piracy Rate',
 'Contributor Page View from: Progress of SIPO Restructuring - Trademark Certificates Stamped with SIPO Seal from June 8th',
 'Contributor Page View from: Unitalen Won Plate Solar Collector Patent Infringement Litigation',
 'Author Page View Lei Zhao',
 'Contributor Page View from: WIPO Report: China‘s Innovation Capacity Continued To Improve',
 'Author Own Biography',
 'Author Own Biography from: Revision Of Anti-Unfair Competition Law Of China']

Now it's clear that the problem arises from activity types information being prefixed to the article titles of these records.

In [179]:
# Define a function to find matching artile title
# if x contains the article title found in `articles` dataset, add the article title to result list
def searchTitle(x, resultlist):
    searchResult = [name for name in articles["Article Title"] if str(x).find(str(name)) >= 0]
    # if match found
    if len(searchResult) != 0:
        # add the matching title from `articles` to result list
        resultlist.append(searchResult[0])
    else:
        # if no match found, add '0' to result list
        resultlist.append(0)
        

# Now, search matching titles for unregistered users
result = [] # intiate the result list
unregUsr_dat["Article Title"].apply(searchTitle, resultlist=result)
unregUsr_dat["Article Title2"] = result # save result to a new column

In [180]:
# check those without matching title found
unregUsr_dat.loc[unregUsr_dat['Article Title2']==0, ['Article Title', 'Activity Type']]

,Article Title,Activity Type
47,Author Page View Unitalen Attorneys At Law,Author Page View
773,Author Page View Lei Zhao,Author Page View
1120,Author Own Biography,Author Bio
1124,Author Page View Lei Zhao,Author Page View
1125,Author Page View Lei Zhao,Author Page View
...,...,...
5604,Google Search on Contributor,Google Search on Contributor
5605,Google Search on Contributor,Google Search on Contributor
5606,Google Search on Contributor,Google Search on Contributor
5607,Google Search on Contributor,Google Search on Contributor


Apparently, the activities in these records are not generated through any articles from our websites. Therefore, they don't have matching article titles from `articles`.

In [181]:
# Do the same for registered users
result = []
regUsr_dat["Article Title"].apply(searchTitle, resultlist=result)
regUsr_dat.loc[:, "Article Title2"] = result

In [182]:
# check those without matching title found
regUsr_dat[regUsr_dat["Article Title2"]==0]

,Article Title,Activity Type,Date Of Activity,Article Primary Author,Article Title2
556,NaN,Search of your Data,2018-05-09,NaN,0


The activity in this record is also not generated through any article. That's why there is no article title. 

# Combine User Activities

In [183]:
# combine registered and unregistered users activities
df = pd.concat([regUsr_dat, unregUsr_dat], ignore_index=True)
df.head(1)

,Article Title,Activity Type,Date Of Activity,Article Primary Author,Article Title2
0,A Successful Trade Dress Protection Case Throu...,Article Read from Email Subscription,2018-02-26,Dan Chen,A Successful Trade Dress Protection Case Throu...


# Group Activities into Bigger Categories

In [184]:
# all unique 'activity type' after removing white space and using upper case
df["Activity Type"] = df["Activity Type"].str.strip().str.upper()
sorted(list(df["Activity Type"].unique()))

['ARTICLE',
 'ARTICLE ADDED TO POCKET',
 'ARTICLE READ FROM EMAIL SUBSCRIPTION',
 'ARTICLE VIEW FROM WEBSITE',
 'AUTHOR BIO',
 'AUTHOR PAGE VIEW',
 'CONTACT DETAILS REQUEST',
 'CONTRIBUTOR PAGE VIEW',
 'CONTRIBUTOR PAGE VIEW FROM ARTICLE',
 'FIRM WEBLINK',
 'FIRM WEBLINK FROM ARTICLE',
 'FORWARDED ARTICLE TO COLLEAGUE',
 'GOOGLE SEARCH ON AUTHOR',
 'GOOGLE SEARCH ON CONTRIBUTOR',
 'IN-ARTICLE CLICK-THROUGH TO YOUR WEBSITE',
 'PRESS RELEASE VIEW',
 'PRESS RELEASE VIEW FROM ARTICLE',
 'PRINTED ARTICLE',
 'REDIRECT TO YOUR WEBSITE FROM ARTICLE',
 'REFERRED BY COLLEAGUE TO THIS ARTICLE',
 'SEARCH OF YOUR DATA',
 'SOCIALMEDIA ARTICLE SHARE',
 'VISIT TO YOUR EVENT INFO PAGE FROM ARITCLE',
 'YOUR HOME PAGE']

In [185]:
# define category groups for activities
category = {'ARTICLE ADDED TO POCKET':'对文章表现出兴趣',
       'FORWARDED ARTICLE TO COLLEAGUE':'对文章表现出兴趣',
       'SOCIALMEDIA ARTICLE SHARE':'对文章表现出兴趣',
       'CONTACT DETAILS REQUEST':'对集佳或作者表现出兴趣',
       'PRINTED ARTICLE':'对集佳或作者表现出兴趣',
       'GOOGLE SEARCH ON AUTHOR':'对集佳或作者表现出兴趣',
       'SEARCH OF YOUR DATA':'对集佳或作者表现出兴趣',
       'GOOGLE SEARCH ON AUTHOR':'对集佳或作者表现出兴趣',
       'ARTICLE':'文章阅读',
       'REFERRED BY COLLEAGUE TO THIS ARTICLE':'文章阅读',
       'ARTICLE VIEW FROM WEBSITE':'文章阅读',
       'ARTICLE READ FROM EMAIL SUBSCRIPTION':'文章阅读',
       'AUTHOR BIO':'对集佳或作者表现出兴趣',
       'AUTHOR PAGE VIEW':'对集佳或作者表现出兴趣',
       'PRESS RELEASE VIEW':'查看我们更多文章',
       'REDIRECT TO YOUR WEBSITE FROM ARTICLE':'对集佳或作者表现出兴趣',
       'YOUR HOME PAGE':'对集佳或作者表现出兴趣',
       'VISIT TO YOUR EVENT INFO PAGE FROM ARITCLE':'对集佳或作者表现出兴趣',
       'CONTRIBUTOR PAGE VIEW':'对集佳或作者表现出兴趣',
       'FIRM WEBLINK FROM ARTICLE':'对集佳或作者表现出兴趣'
}

In [186]:
# match category group by activity type for one record
category.get(df["Activity Type"][0])

'文章阅读'

In [187]:
# match category groups by activity types for all records
result = [category.get(x) for x in df["Activity Type"]]
df["category"] = result
df.head(1)

,Article Title,Activity Type,Date Of Activity,Article Primary Author,Article Title2,category
0,A Successful Trade Dress Protection Case Throu...,ARTICLE READ FROM EMAIL SUBSCRIPTION,2018-02-26,Dan Chen,A Successful Trade Dress Protection Case Throu...,文章阅读


# Counts by Activity, Category Group

In [188]:
df["category"].value_counts()

文章阅读           10446
对集佳或作者表现出兴趣      400
对文章表现出兴趣           6
查看我们更多文章           1
Name: category, dtype: int64

In [189]:
count_df = df.groupby(['category', 'Activity Type']).size().to_frame()
count_df

0
category    Activity Type                                   
对文章表现出兴趣    ARTICLE ADDED TO POCKET                        2
            FORWARDED ARTICLE TO COLLEAGUE                 3
            SOCIALMEDIA ARTICLE SHARE                      1
对集佳或作者表现出兴趣 AUTHOR BIO                                    18
            AUTHOR PAGE VIEW                              31
            CONTACT DETAILS REQUEST                        2
            CONTRIBUTOR PAGE VIEW                          5
            FIRM WEBLINK FROM ARTICLE                     58
            GOOGLE SEARCH ON AUTHOR                        4
            PRINTED ARTICLE                               49
            REDIRECT TO YOUR WEBSITE FROM ARTICLE        187
            SEARCH OF YOUR DATA                            4
            VISIT TO YOUR EVENT INFO PAGE FROM ARITCLE    36
            YOUR HOME PAGE                                 6
文章阅读        ARTICLE                                     9964
            ARTICLE READ FROM EMAIL SUBSCRIPTION         356
            ARTICLE VIEW FROM WEBSITE                    125
            REFERRED BY COLLEAGUE TO THIS ARTICLE          1
查看我们更多文章    PRESS RELEASE VIEW                             1

# Counts by Topic, Articles Types 

In [190]:
# Inner join user activity and articles datasets on `Article Title`
# only include records that have matching values in both datasets
df = df.merge(articles, on='Article Title')
df.rename(columns={'Matter of Concern':'Topic'}, inplace=True)
df.head(1)

,Article Title,Activity Type,Date Of Activity,Article Primary Author,Article Title2,category,Article Type,Topic
0,A Successful Trade Dress Protection Case Throu...,ARTICLE READ FROM EMAIL SUBSCRIPTION,2018-02-26,Dan Chen,A Successful Trade Dress Protection Case Throu...,文章阅读,Unitalen Cases,Trade Dress


In [191]:
df["Article Type"].value_counts().to_frame()

,Article Type
IP News,4338
Unitalen Cases,4286
Unitalen,1484
Attorney Article,390


In [192]:
pivot_tb = pd.pivot_table(df, values=['Article Title'], index=['Article Type', 'Topic'], 
              aggfunc={'Article Title':['nunique', 'count']})
pivot_tb.columns = ['Clicks', 'Number of Articles']
pivot_tb

Clicks  \
Article Type     Topic                                        
Attorney Article Joint Ventures                         289   
                 Trademark Law                          101   
IP News          Anti-Unfair Competition Law            526   
                 China IP System & Policy              1750   
                 China-US                               190   
                 E-Commerce Law                         482   
                 Invention Patent                        22   
                 Patent Application                     545   
                 Piracy                                 582   
                 Trademark Case                          41   
                 Trademark Law                           39   
                 Trademark Review                       161   
Unitalen         Awards & Honors                       1457   
                 Important Visitors                      27   
Unitalen Cases   Patent Infringement                    351   
                 Patent Invalidation                    459   
                 Pre-Trial Behavioral Preservation      386   
                 Settlement                             463   
                 Time-honored Brand                     267   
                 Trade Dress                            539   
                 Trademark Case with High Indemnity     324   
                 Trademark Review                        79   
                 Well-known Trademark                  1418   

                                                     Number of Articles  
Article Type     Topic                                                   
Attorney Article Joint Ventures                                       1  
                 Trademark Law                                        1  
IP News          Anti-Unfair Competition Law                          1  
                 China IP System & Policy                             7  
                 China-US                                             1  
                 E-Commerce Law                                       1  
                 Invention Patent                                     1  
                 Patent Application                                   1  
                 Piracy                                               1  
                 Trademark Case                                       1  
                 Trademark Law                                        1  
                 Trademark Review                                     1  
Unitalen         Awards & Honors                                      2  
                 Important Visitors                                   1  
Unitalen Cases   Patent Infringement                                  1  
                 Patent Invalidation                                  1  
                 Pre-Trial Behavioral Preservation                    1  
                 Settlement                                           1  
                 Time-honored Brand                                   1  
                 Trade Dress                                          1  
                 Trademark Case with High Indemnity                   1  
                 Trademark Review                                     2  
                 Well-known Trademark                                 2